In [2]:
import numpy as np
import pandas as pd
import re
import dateutil.parser as parser

In [ ]:
def is_datetime(containsDates):
    """
    Inputs: a string type that may represent the date/time that a given transaction took place
    Outputs: A boolean that describes whether the input is a datetime
    """
    everyDatePattern = {
    '%Y/%m/%d': r'^\s*\d{2,4}(\/|-|\.|\s)([a-yA-Y]{3}\.{0, 1}|\d{1,2})(\/|-|\.|\s)\d{1,2}\s*$',
    '%d/%m/%Y': r'^\s*\d{1,2}(\/|-|\.|\s)([a-yA-Y]{3}|\.{0,1})\d{1,2}(\/|-|\.|\s)\d{2,4}\s*$',
    '%m/%d/%Y': r'^\s*([a-yA-Y]{3}\.{0,1}|\d{1,2})(\/|-|\.|\s)\d{1,2}(\/|-|\.|\s)\d{2,4}\s*$'}

    dateType = {}
    for datePattern in everyDatePattern.keys():
        possibleDates = re.findall(everyDatePattern[datePattern], containsDates)
        #alter each possible date format to match the date format of users.csv
        dateType[datePattern] = possibleDates 

    if len(possibleDates) == 0:
        return "No Date"

    ## check what date type it is with year, day or month first

    dateType = {}
    for datePatternRegex,matchedDates in dateType.items():
        for date in dateArray:
            dateType[date] = datePatternRegex

    for date in dateType.keys():
        if dateType[date] == '%Y/%m/%d':
            parser.parse(date, yearfirst = True)
        if dateType[date] == '%d/%m/%Y':
            parser.parse(date, dayfirst= True)


    if len(possibleDates) != 0:
        first = parser.parse(possibleDates[0])
        last = parser.parse(possibleDates[-1])
        return [first, last]






In [91]:
def getTotal(lineItemArray):
    """
    Input: a list ordered moving down the page of y-coordinates with 
    entries of the associated line item
    Output: float type: the determined total
    """
    
    lineItemArrayTyped = []
    for lineItem in lineItemArray:
        lineItemArrayTyped.append(abs(float(lineItem)))

    #print(lineItemArrayTyped)

    assumedTotal = max(lineItemArrayTyped)
    #print(assumedTotal)

    for lineItem in lineItemArrayTyped:
        #print("lineItem = ", lineItem)
        if lineItem != assumedTotal:
            otherLineItems = lineItemArrayTyped.copy()
            otherLineItems.remove(lineItem)
            if assumedTotal - lineItem in otherLineItems:
                #print("assumedTotal - lineItem =", assumedTotal - lineItem )
                lineItemArrayTyped.reverse()
                assumedTotalIdx = lineItemArrayTyped.index(assumedTotal) 
                #print("assumedTotalIdx", assumedTotalIdx, lineItemArrayTyped[assumedTotalIdx])
                lineItemIdx = lineItemArrayTyped.index(lineItem)
                #print("lineItemIdx", lineItemIdx, lineItemArrayTyped[lineItemIdx])
                changeIdx = lineItemArrayTyped.index(assumedTotal - lineItem)
                #print("changeIdx", changeIdx, lineItemArrayTyped[changeIdx])
                if (changeIdx < assumedTotalIdx < lineItemIdx) or (lineItemIdx < assumedTotalIdx < changeIdx):
                    #print("Change Tendered")
                    lineItemArrayTyped[assumedTotalIdx] = 0
                    #print("array w/ removed assumed total", lineItemArrayTyped)
                    if changeIdx > assumedTotalIdx:
                        #print("changeIdx", changeIdx)
                        #print("assumedTotalIdx", assumedTotalIdx)
                        lineItemArrayTyped.remove(lineItemArrayTyped[lineItemIdx])
                        #print("change removed", lineItemArrayTyped)
                    else:
                        lineItemArrayTyped.remove(lineItemArrayTyped[changeIdx])

                    return max(lineItemArrayTyped)

    return assumedTotal



In [97]:
testCasesForGetTotal = pd.read_csv("price_train (2).csv")


#print(getTotal([2, 3, 5]))
#print(getTotal([2, 5, 8, 3]))
#print(getTotal(['9.90', '38.00', '0.20', '48.10', '50.20', '2.10', '2.72']))

for i, priceCase in testCasesForGetTotal.iterrows():
    docId, row = priceCase
    row = eval(row)
    print(docId, row)
    print(getTotal(row))
    




00d0243046961 ['7.00', '2.65', '9.65', '-50.00', '40.35', '9.10', '0.55', '0.00']
9.65
00d0164592408 ['2.80', '5.60', '0.40', '0.20', '5.66', '0.34', '6.00', '6.00', '6.00', '5.66', '0.34']
6.0
00d0696518680 ['0.70', '7.00', '10.00', '7.00', '7.00', '7.00', '0.00', '6.60', '0.40']
10.0
00d0142736377 ['15.00', '5.00', '15.00', '35.00', '35.00', '35.00', '35.00', '35.00', '0.00']
35.0
00d0530724847 ['5.60', '2.80', '0.40', '0.20', '5.66', '0.34', '6.00', '6.00', '6.00', '5.66', '0.34']
6.0
00d0114666542 ['4.40', '2.20', '0.20', '0.40', '4.80', '4.80', '4.80', '4.80', '4.80', '0.00']
4.8
00d0770154989 ['1.00', '15.90', '15.90', '15.90', '15.00', '15.90', '15.90', '50.00', '34.10', '15.00', '0.90']
15.9
00d0201817639 ['19.78', '12.00', '-1.20', '20.90', '-2.09', '49.39', '0.01', '49.40', '-50.00', '0.60', '2.18', '36.41', '0.00', '10.80', '-3.29']
49.4
00d0607552804 ['4.00', '8.00', '3.5', '8.00', '0.00', '8.00', '8.00', '7.55', '0.45']
8.0
00d0602427476 ['4.40', '4.40', '0.50', '0.50', '3

In [ ]:
def searchGetTotal(arrayOfCoords, givenPoint):
    """
    Inputs: 
        arrayOfCoords: An array with tuples representing (x, y) 
                        coordinates of all the unknown lineItems in a receipt
        givenPoint: tuple representing the coordinates of the NER labeled 
                    total amount
    Outputs: Coordinates of the point closest to givenPoint in array of Coords
    """
    bestDistance = 10000000000000000
    bestCoord = None
    for coordinatePair in arrayOfCoords:
        distance = ((givenPoint[0] - coordinatePair[0]) ** 2 + (givenPoint[1] - coordinatePair[1]) ** 2) ** (1/2)
        if distance < bestDistance:
            bestDistance = distance
            bestCoord = coordinatePair
        
    return bestCoord
        
    
    
    

    